In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
# CONSTANTES

URLS = [
    'https://www.c5n.com/deportes/mundial-sub-17-argentina-cayo-frente-alemania-y-jugara-el-tercer-puesto-n136601',
    'https://www.c5n.com/politica/javier-milei-se-reune-jake-sullivan-asesor-seguridad-nacional-joe-biden-n136598',
    'https://www.c5n.com/economia/dolar-blue-hoy-cuanto-cotiza-el-28-noviembre-n136604',
    'https://www.c5n.com/astrologia/horoscopo-hoy-martes-28-noviembre-n136580',
    'https://www.c5n.com/tecnologia/alerta-android-una-app-mensajes-espia-tus-datos-n136409',
    'https://www.c5n.com/ratingcero/emotivo-taylor-swift-recibio-la-familia-la-fanatica-que-murio-brasil-n136557'
]

In [3]:
class NewsPipeline:

    def __init__(self):

        self.KEYWORDS = ['article', 'noticias', 'news', 'nota', '/']
        self.PALABRAS_CLAVES = {
            'salud': ['salud', 'medicina', 'enfermedad', 'medico', 'bienestar'],
            'deportes': ['deportes', 'futbol', 'baloncesto', 'tenis', 'competicion'],
            'medioambiente': ['medioambiente', 'ecologia', 'naturaleza', 'sostenibilidad', 'cambio climatico'],
            'economia': ['economia', 'finanzas', 'negocios', 'mercado', 'inversion'],
            'politica': ['politica', 'gobierno', 'elecciones', 'legislación', 'parlamento'],
            'entretenimiento': ['entretenimiento', 'cine', 'musica', 'celebridades', 'espectaculos'],
            'horoscopo': ['horoscopo', 'astrologia', 'signos zodiacales', 'predicciones', 'astrologia diaria'],
            'cultura': ['cultura', 'arte', 'literatura', 'tradiciones', 'patrimonio'],
            'ciencia_tecnologia': ['ciencia', 'tecnologia', 'innovacion', 'descubrimientos', 'avances cientificos'],
            'educacion': ['educacion', 'escuela', 'aprendizaje', 'docentes', 'formacion']
        }

        self.fechas = []
        
        self.textos = []
        
        self.titulos = []
        
        self.categorias = []

    def determinar_categoria(self, texto_noticia):
        
        for categoria, palabras in self.PALABRAS_CLAVES.items():
            
            for palabra in palabras:
                
                if palabra in texto_noticia.lower():
                    
                    return categoria
                    
        return "OTRA"

    def scrape_noticia(self, url):
        
        response = requests.get(url)
        
        soup = BeautifulSoup(response.content, "html.parser")

        titulo_noticia = soup.find("h1").text.strip() if soup.find("h1") else "NO ESPECIFICADO"

        texto_noticia = " ".join([p.text.strip() for p in soup.find_all("p")])

        categoria = self.determinar_categoria(texto_noticia)

        fecha_noticia = soup.find(["time", "span", "p", "publication-date", "fecha"], {"datetime": True})
        
        fecha_noticia = fecha_noticia["datetime"].strip() if fecha_noticia else "FECHA NO ENCONTRADA"

        return titulo_noticia, texto_noticia, categoria, fecha_noticia

    def create_csv(self, urls_list):

        for url in urls_list:

            result = self.scrape_noticia(url)
            
            self.fechas.append(result[3])
            
            self.textos.append(result[1])
            
            self.titulos.append(result[0])
            
            self.categorias.append(result[2])

        data = {
            'fecha_noticia': self.fechas,
            'titulo_noticia': self.titulos,
            'texto_noticia': self.textos,
            'categoria': self.categorias
        }

        df = pd.DataFrame(data)

        df['longitud_noticia'] = df['texto_noticia'].apply(len)
        
        df = df[['fecha_noticia', 'titulo_noticia', 'texto_noticia', 'longitud_noticia', 'categoria']]

        df = df.sort_values(by=['categoria', 'fecha_noticia'])

        output_file = 'data_news.xlsx'
        
        df.to_excel(output_file, index=False, engine='xlsxwriter')

        print(df)

        # Imprimir mensaje de confirmación
        print(f"Los datos se han guardado en '{output_file}'.")

    

In [4]:
npipe = NewsPipeline()

In [5]:
npipe.create_csv(URLS)

              fecha_noticia  \
5  2023-11-27T21:14:44.000Z   
0  2023-11-28T10:27:00.000Z   
2  2023-11-28T11:16:00.000Z   
4  2023-11-27T13:57:17.000Z   
1  2023-11-28T10:33:00.000Z   
3  2023-11-27T23:58:55.000Z   

                                      titulo_noticia  \
5  Emotivo: Taylor Swift recibió a la familia de ...   
0  Mundial Sub 17: Argentina cayó frente a Aleman...   
2  Dólar blue hoy: a cuánto cotiza el 28 de novie...   
4  Alerta Android: una app de mensajes espía tus ...   
1  Milei se reunió con Jake Sullivan, asesor de B...   
3           Horóscopo de hoy, martes 28 de noviembre   

                                       texto_noticia  longitud_noticia  \
5  Ana Clara Benavides falleció en la previa al p...              2341   
0  Los dirigidos por Diego Placente lograron igua...              4056   
2  El tipo de cambio informal bajó $45 y cerró a ...              3632   
4  La aplicación fue retirada de la tienda oficia...              3284   
1  El presidente el